In [135]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import numpy as np

In [ ]:
## TODO: convert date range to datetime column
## TODO: time series prediction for next X days (say next month)

In [39]:
obsidian_flames = pd.read_csv('Obsidian_Flames.csv', index_col = 'Unnamed: 0')
obsidian_flames

,Product,Foil,Date Range,Market Price
0,Absol ex - 135/197,['Holofoil Market Price: '],7/18 to 7/20,$0.00
1,Absol ex - 135/197,['Holofoil Market Price: '],7/21 to 7/23,$0.00
2,Absol ex - 135/197,['Holofoil Market Price: '],7/24 to 7/26,$0.00
3,Absol ex - 135/197,['Holofoil Market Price: '],7/27 to 7/29,$0.00
4,Absol ex - 135/197,['Holofoil Market Price: '],7/30 to 8/1,$4.87
...,...,...,...,...
12925,Zigzagoon,['Reverse Holofoil Market Price: '],10/7 to 10/9,$0.06
12926,Zigzagoon,[' Market Price: '],10/10 to 10/12,$0.03
12927,Zigzagoon,['Reverse Holofoil Market Price: '],10/10 to 10/12,$0.06
12928,Zigzagoon,[' Market Price: '],10/13 to 10/15,$0.03


In [40]:
obsidian_flames['Foil'].unique()

array(["['Holofoil Market Price: ']", "[' Market Price: ']",
       "['Reverse Holofoil Market Price: ']"], dtype=object)

In [41]:
obsidian_flames.replace({'Foil' : { "[' Market Price: ']" : "Normal",
                               "['Reverse Holofoil Market Price: ']" : "Reverse Holofoil", 
                               "['Holofoil Market Price: ']" : "Holofoil"}}, inplace=True)
obsidian_flames

,Product,Foil,Date Range,Market Price
0,Absol ex - 135/197,Holofoil,7/18 to 7/20,$0.00
1,Absol ex - 135/197,Holofoil,7/21 to 7/23,$0.00
2,Absol ex - 135/197,Holofoil,7/24 to 7/26,$0.00
3,Absol ex - 135/197,Holofoil,7/27 to 7/29,$0.00
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,$4.87
...,...,...,...,...
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,$0.06
12926,Zigzagoon,Normal,10/10 to 10/12,$0.03
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,$0.06
12928,Zigzagoon,Normal,10/13 to 10/15,$0.03


In [42]:
sum(obsidian_flames['Market Price'] == "$0.00") / len(obsidian_flames['Market Price'] == "$0.00")
# Just 20% of the dataset has $0.00 market price

0.20471771075019335

In [43]:
of_cleaned = obsidian_flames[obsidian_flames['Market Price'] != '$0.00']
of_cleaned

,Product,Foil,Date Range,Market Price
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,$4.87
5,Absol ex - 135/197,Holofoil,8/2 to 8/4,$4.74
6,Absol ex - 135/197,Holofoil,8/5 to 8/7,$4.74
7,Absol ex - 135/197,Holofoil,8/8 to 8/10,$2.04
8,Absol ex - 135/197,Holofoil,8/11 to 8/13,$0.80
...,...,...,...,...
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,$0.06
12926,Zigzagoon,Normal,10/10 to 10/12,$0.03
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,$0.06
12928,Zigzagoon,Normal,10/13 to 10/15,$0.03


In [44]:
of_cleaned = of_cleaned.sort_values(by=['Product', 'Foil'])
of_cleaned

,Product,Foil,Date Range,Market Price
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,$4.87
5,Absol ex - 135/197,Holofoil,8/2 to 8/4,$4.74
6,Absol ex - 135/197,Holofoil,8/5 to 8/7,$4.74
7,Absol ex - 135/197,Holofoil,8/8 to 8/10,$2.04
8,Absol ex - 135/197,Holofoil,8/11 to 8/13,$0.80
...,...,...,...,...
12921,Zigzagoon,Reverse Holofoil,10/1 to 10/3,$0.06
12923,Zigzagoon,Reverse Holofoil,10/4 to 10/6,$0.06
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,$0.06
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,$0.06


In [45]:
of_cleaned['Market Price'] = of_cleaned['Market Price'].replace({'\$':''}, regex = True)
of_cleaned

,Product,Foil,Date Range,Market Price
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,4.87
5,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74
6,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74
7,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04
8,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80
...,...,...,...,...
12921,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06
12923,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06


In [46]:
of_cleaned = of_cleaned.reset_index(drop=True)
of_cleaned

,Product,Foil,Date Range,Market Price
0,Absol ex - 135/197,Holofoil,7/30 to 8/1,4.87
1,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74
2,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74
3,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04
4,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80
...,...,...,...,...
10278,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06
10279,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06
10280,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06
10281,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06


In [47]:
# tells us whether the product is the same as the previous line product
match_prev_product = [of_cleaned['Product'][i] == of_cleaned['Product'][i+1] for i in range(len(of_cleaned['Product'])-1)]
match_prev_foil = [of_cleaned['Foil'][i] == of_cleaned['Foil'][i+1] for i in range(len(of_cleaned['Foil'])-1)]
prev_market_price = of_cleaned['Market Price'][:-1]
match_prev_product and match_prev_foil

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 T

In [48]:
prev_market_price

0        4.87
1        4.74
2        4.74
3        2.04
4        0.80
         ... 
10277    0.06
10278    0.06
10279    0.06
10280    0.06
10281    0.06
Name: Market Price, Length: 10282, dtype: object

In [49]:
of_cleaned = of_cleaned[1:]
of_cleaned = of_cleaned.reset_index(drop=True)
of_cleaned['Previous Market Price'] = prev_market_price
of_cleaned

,Product,Foil,Date Range,Market Price,Previous Market Price
0,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74,4.87
1,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74,4.74
2,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04,4.74
3,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80,2.04
4,Absol ex - 135/197,Holofoil,8/14 to 8/16,0.86,0.80
...,...,...,...,...,...
10277,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06,0.06
10278,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06,0.06
10279,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06,0.06
10280,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06,0.06


In [50]:
prev_match = match_prev_product and match_prev_foil

In [53]:
of_cleaned = of_cleaned[prev_match]
of_cleaned = of_cleaned.reset_index(drop=True)
of_cleaned

,Product,Foil,Date Range,Market Price,Previous Market Price
0,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74,4.87
1,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74,4.74
2,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04,4.74
3,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80,2.04
4,Absol ex - 135/197,Holofoil,8/14 to 8/16,0.86,0.80
...,...,...,...,...,...
9892,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06,0.06
9893,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06,0.06
9894,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06,0.06
9895,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06,0.06


# Data Split

In [57]:
of_cleaned['Date Range'].unique()

array(['8/2 to 8/4', '8/5 to 8/7', '8/8 to 8/10', '8/11 to 8/13',
       '8/14 to 8/16', '8/17 to 8/19', '8/20 to 8/22', '8/23 to 8/25',
       '8/26 to 8/28', '8/29 to 8/31', '9/1 to 9/3', '9/4 to 9/6',
       '9/7 to 9/9', '9/10 to 9/12', '9/13 to 9/15', '9/16 to 9/18',
       '9/19 to 9/21', '9/22 to 9/24', '9/25 to 9/27', '9/28 to 9/30',
       '10/1 to 10/3', '10/4 to 10/6', '10/7 to 10/9', '10/10 to 10/12',
       '10/13 to 10/15', '7/30 to 8/1', '7/21 to 7/23', '7/24 to 7/26',
       '7/27 to 7/29', '7/18 to 7/20'], dtype=object)

In [108]:
# test set will contain everything from September 25th on
test = of_cleaned.loc[of_cleaned['Date Range'].isin(['9/25 to 9/27', '9/28 to 9/30',
       '10/1 to 10/3', '10/4 to 10/6', '10/7 to 10/9', '10/10 to 10/12', '10/13 to 10/15'])]
train = of_cleaned.loc[of_cleaned['Date Range'].isin(['7/30 to 8/1', '7/21 to 7/23', '7/24 to 7/26',
       '7/27 to 7/29', '7/18 to 7/20', '8/2 to 8/4', '8/5 to 8/7', '8/8 to 8/10', '8/11 to 8/13',
       '8/14 to 8/16', '8/17 to 8/19', '8/20 to 8/22', '8/23 to 8/25',
       '8/26 to 8/28', '8/29 to 8/31', '9/1 to 9/3', '9/4 to 9/6',
       '9/7 to 9/9', '9/10 to 9/12', '9/13 to 9/15', '9/16 to 9/18',
       '9/19 to 9/21', '9/22 to 9/24'])]
test

,Product,Foil,Date Range,Market Price,Previous Market Price
18,Absol ex - 135/197,Holofoil,9/25 to 9/27,1.02,1.08
19,Absol ex - 135/197,Holofoil,9/28 to 9/30,1.16,1.02
20,Absol ex - 135/197,Holofoil,10/1 to 10/3,1.16,1.16
21,Absol ex - 135/197,Holofoil,10/4 to 10/6,1.45,1.16
22,Absol ex - 135/197,Holofoil,10/7 to 10/9,1.52,1.45
...,...,...,...,...,...
9892,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06,0.06
9893,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06,0.06
9894,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06,0.06
9895,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06,0.06


In [109]:
train

,Product,Foil,Date Range,Market Price,Previous Market Price
0,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74,4.87
1,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74,4.74
2,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04,4.74
3,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80,2.04
4,Absol ex - 135/197,Holofoil,8/14 to 8/16,0.86,0.80
...,...,...,...,...,...
9885,Zigzagoon,Reverse Holofoil,9/10 to 9/12,0.09,0.10
9886,Zigzagoon,Reverse Holofoil,9/13 to 9/15,0.08,0.09
9887,Zigzagoon,Reverse Holofoil,9/16 to 9/18,0.07,0.08
9888,Zigzagoon,Reverse Holofoil,9/19 to 9/21,0.06,0.07


In [107]:
of_cleaned.iloc[5884]

Product                  Obsidian Flames Build &amp; Battle Box Display
Foil                                                             Normal
Date Range                                                 9/22 to 9/24
Market Price                                                     121.98
Previous Market Price                                             14.48
Name: 5884, dtype: object

In [123]:
X_test = test[['Product', 'Foil', 'Previous Market Price']]
y_test = test['Market Price'].astype(float)
X_train = train[['Product', 'Foil', 'Previous Market Price']]
y_train = train['Market Price'].astype(float)

In [124]:
X_test = pd.get_dummies(X_test, columns = ['Product', 'Foil'])
X_train = pd.get_dummies(X_train, columns = ['Product', 'Foil'])

# Linear Regression

In [125]:
classifier = LinearRegression()
classifier.fit(X_train, y_train)

LinearRegression()

In [126]:
coefficients = pd.DataFrame(zip(X_train.columns, classifier.coef_))
coefficients

,0,1
0,Previous Market Price,1.056441e-01
1,Product_Absol ex - 135/197,-3.490604e+05
2,Product_Absol ex - 214/197,-3.490599e+05
3,Product_Altaria,-3.490616e+05
4,Product_Amoonguss,-3.490616e+05
...,...,...
253,Product_Yungoos,-3.490616e+05
254,Product_Zigzagoon,-3.490616e+05
255,Foil_Holofoil,-1.101045e+09
256,Foil_Normal,-1.101045e+09


In [127]:
y_pred = classifier.predict(X_test)

In [136]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
#     mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
#     print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [137]:
regression_results(y_test, y_pred)
# 98% of variance explained which makes sense given that it has access to the previous value

explained_variance:  0.9838
r2:  0.9835
MAE:  0.6744
MSE:  18.5895
RMSE:  4.3116
